In [14]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [12]:
import requests
from bs4 import BeautifulSoup

# URL of the Webpage to scrape
url = "https://www.promptingguide.ai/research/llm-agents"

# Headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a GET request to the URL
response = requests.get(url, headers=headers, timeout=10)

print(response)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extract the desired information from the HTML
    # For example, to get the repository name
    webpage_content = soup.get_text()
    
    # Print the repository name
    print("Webpage content:\n\n", webpage_content)
else:
    print("Failed to retrieve the webpage")

<Response [200]>
Webpage content:

 LLM Agents | Prompt Engineering Guide Prompt Engineering Guide🎓 Prompt Engineering Course🎓 Prompt Engineering CourseServicesServicesAboutAboutGitHubGitHub (opens in a new tab)DiscordDiscord (opens in a new tab)Prompt EngineeringIntroductionLLM SettingsBasics of PromptingPrompt ElementsGeneral Tips for Designing PromptsExamples of PromptsPrompting TechniquesZero-shot PromptingFew-shot PromptingChain-of-Thought PromptingMeta PromptingSelf-ConsistencyGenerate Knowledge PromptingPrompt ChainingTree of ThoughtsRetrieval Augmented GenerationAutomatic Reasoning and Tool-useAutomatic Prompt EngineerActive-PromptDirectional Stimulus PromptingProgram-Aided Language ModelsReActReflexionMultimodal CoTGraph PromptingAgentsIntroduction to AgentsAgent ComponentsGuidesOptimizing PromptsApplicationsFine-tuning GPT-4oFunction CallingContext Caching with LLMsGenerating DataGenerating Synthetic Dataset for RAGTackling Generated Datasets DiversityGenerating CodeGraduate 

In [13]:
with open("webpage_content.md", "w") as f:
    f.write(webpage_content)

In [15]:
from openai import OpenAI

client = OpenAI()

In [19]:
file = client.files.create(
    file=open("webpage_content.md", "rb"),
    purpose="assistants"
)

assistant = client.beta.assistants.create(
    name="AI Agents Expert",
    description="You are an expert at AI Agents and can answer questions about them.",
    tools=[{"type": "file_search"}],
    model="gpt-4o",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"


In [20]:
vector_store = client.beta.vector_stores.create(name="AI Agents Research")

file_paths = ["./webpage_content.md"]

file_streams = [open(file_path, "rb") for file_path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)

print(file_batch.id)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/vector_stores "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/vector_stores/vs_EszGI031tZlCkAhq2PYgjdxs/file_batches "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_EszGI031tZlCkAhq2PYgjdxs/file_batches/vsfb_bda2eb04db1c4b18a482adbd92fc976d "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_EszGI031tZlCkAhq2PYgjdxs/file_batches/vsfb_bda2eb04db1c4b18a482adbd92fc976d "HTTP/1.1 200 OK"


completed
vsfb_bda2eb04db1c4b18a482adbd92fc976d


In [21]:
print(file_batch.file_counts)

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [23]:
# Let's update the assistant with the new vector store

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants/asst_3T17byAbqA8Tq6NM1rcu6IQn "HTTP/1.1 200 OK"


In [24]:
# Optionally we could attach the files directly in the thread
# This would create its own vector store if no other vector store is already attached
# to the assistant


# Upload the user provided file to OpenAI
# message_file = client.files.create(
# file=open("edgar/aapl-10k.pdf", "rb"), purpose="assistants"
# )

# # Create a thread and attach the file to the message
# thread = client.beta.threads.create(
# messages=[
#   {
#     "role": "user",
#     "content": "How many shares of AAPL were outstanding at the end of of October 2023?",
#     # Attach the new file to the message.
#     "attachments": [
#       { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
#     ],
#   }
# ]
# )

# # The thread now has a vector store with that file in its tool resources.
# print(thread.tool_resources.file_search)

In [25]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "What are the core components of an LLM Agent framework?"
        }
    ]
)

print(thread.id)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"


thread_RanfLa8BqW08RiJe9qpBeHIN


In [27]:
# Let's create a run and check the output
# First without streaming


run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

print(run)

if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )
    print(messages)
else:
    print(run.status)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/runs "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/runs/run_1TGDfrqBWqCuCNO5MPlNnOzg "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/runs/run_1TGDfrqBWqCuCNO5MPlNnOzg "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/runs/run_1TGDfrqBWqCuCNO5MPlNnOzg "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/runs/run_1TGDfrqBWqCuCNO5MPlNnOzg "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/runs/run_1TGDfrqBWqCuCNO5MPlNnOzg "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/runs/run_1TGDfrqBWqCuCNO5MPlNnOzg "HTTP/1.1 200 OK"

Run(id='run_1TGDfrqBWqCuCNO5MPlNnOzg', assistant_id='asst_3T17byAbqA8Tq6NM1rcu6IQn', cancelled_at=None, completed_at=1736764645, created_at=1736764634, expires_at=None, failed_at=None, incomplete_details=None, instructions=None, last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1736764636, status='completed', thread_id='thread_RanfLa8BqW08RiJe9qpBeHIN', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=274, prompt_tokens=9013, total_tokens=9287, prompt_token_details={'cached_tokens': 0}), temperature=1.0, top_p=1.0, tool_resources={})


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/threads/thread_RanfLa8BqW08RiJe9qpBeHIN/messages "HTTP/1.1 200 OK"


SyncCursorPage[Message](data=[Message(id='msg_e9yfp7Ii5U0H1iIh0mIDql6f', assistant_id='asst_3T17byAbqA8Tq6NM1rcu6IQn', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=630, file_citation=FileCitation(file_id='file-QjhqQcqarzc2zNfGEg3raf'), start_index=618, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=1078, file_citation=FileCitation(file_id='file-QjhqQcqarzc2zNfGEg3raf'), start_index=1066, text='【4:0†source】', type='file_citation')], value="The core components of an LLM Agent framework typically include:\n\n1. **User Request**: The initial input or question posed by the user.\n   \n2. **Agent/Brain**: The central component that acts as the coordinator. It is the large language model (LLM) responsible for processing the user request and orchestrating the overall operations.\n   \n3. **Planning**: A module to assist the agent in planning future actions. It helps in decomposing tasks and cr

In [34]:
messages.data[0].content[0].text.value

"The core components of an LLM Agent framework typically include:\n\n1. **User Request**: The initial input or question posed by the user.\n   \n2. **Agent/Brain**: The central component that acts as the coordinator. It is the large language model (LLM) responsible for processing the user request and orchestrating the overall operations.\n   \n3. **Planning**: A module to assist the agent in planning future actions. It helps in decomposing tasks and creating a detailed plan of subtasks necessary to address the user request. Planning can be enhanced with feedback to iteratively refine the plan based on past performance【4:0†source】.\n\n4. **Memory**: This component manages the agent's past behaviors and interactions. It typically includes two types of memory:\n   - **Short-term Memory**: Contextual information about the agent's current situation, often hindered by context window constraints.\n   - **Long-term Memory**: Retention and recall of past behaviors and experiences over an extend

In [ ]:
# Example where we include file citations

# run = client.beta.threads.runs.create_and_poll(
#   thread_id=thread.id, assistant_id=assistant.id
# )

# messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

# message_content = messages[0].content[0].text
# annotations = message_content.annotations
# citations = []
# for index, annotation in enumerate(annotations):
#   message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
#   if file_citation := getattr(annotation, "file_citation", None):
#       cited_file = client.files.retrieve(file_citation.file_id)
#       citations.append(f"[{index}] {cited_file.filename}")

# print(message_content.value)
# print("\n".join(citations))